In [10]:
import os
import re
from collections import defaultdict

# Path to the folder containing the JSONL files
folder_path = "experiment_data"

# Corrected pattern to match filenames and extract information
filename_pattern = re.compile(
    r"(?P<transmitter>\w+)_"
    r"(?P<blocker>\w+)_"
    r"(?P<tx_power>-?\d+)dBm(?P<block_power>-?\d+)dBm_"
    r"(?P<tx_freq>\d+)MHz(?P<block_freq>\d+)MHz"
)

# Dictionary to store grouped data by transmitter_blocker
grouped_files = defaultdict(list)

# Parse filenames in the folder
for filename in os.listdir(folder_path):
    match = filename_pattern.match(filename)
    if match:
        data = match.groupdict()
        # Extract and convert data
        data["tx_power"] = int(data["tx_power"])
        data["block_power"] = int(data["block_power"])
        data["tx_freq"] = int(data["tx_freq"])
        data["block_freq"] = int(data["block_freq"])
        data["frequency_offset"] = data["tx_freq"] - data["block_freq"]
        data["filename"] = filename

        # Group by transmitter_blocker
        transmitter_blocker = f"{data['transmitter']}_{data['blocker']}"
        grouped_files[transmitter_blocker].append(data)

# Sort each group first by frequency offset, then by blocker power
for transmitter_blocker, files in grouped_files.items():
    # First sort by frequency offset
    files.sort(key=lambda x: x["frequency_offset"])
    # Then within each frequency offset, sort by blocker power
    files.sort(key=lambda x: (x["frequency_offset"], x["block_power"]))

# Print the grouped and sorted data for validation
for transmitter_blocker, files in grouped_files.items():
    print(f"Group: {transmitter_blocker}, length: {len(files)}")
    for file in files:
        print(file)
    print("\n")


Group: BLE1MBit_tone, length: 35
{'transmitter': 'BLE1MBit', 'blocker': 'tone', 'tx_power': 0, 'block_power': -8, 'tx_freq': 2425, 'block_freq': 2427, 'frequency_offset': -2, 'filename': 'BLE1MBit_tone_0dBm-8dBm_2425MHz2427MHz_delay255us_tx120B_block645us.jsonl'}
{'transmitter': 'BLE1MBit', 'blocker': 'tone', 'tx_power': 0, 'block_power': -4, 'tx_freq': 2425, 'block_freq': 2427, 'frequency_offset': -2, 'filename': 'BLE1MBit_tone_0dBm-4dBm_2425MHz2427MHz_delay255us_tx120B_block645us.jsonl'}
{'transmitter': 'BLE1MBit', 'blocker': 'tone', 'tx_power': 0, 'block_power': 0, 'tx_freq': 2425, 'block_freq': 2427, 'frequency_offset': -2, 'filename': 'BLE1MBit_tone_0dBm0dBm_2425MHz2427MHz_delay255us_tx120B_block645us.jsonl'}
{'transmitter': 'BLE1MBit', 'blocker': 'tone', 'tx_power': 0, 'block_power': 2, 'tx_freq': 2425, 'block_freq': 2427, 'frequency_offset': -2, 'filename': 'BLE1MBit_tone_0dBm2dBm_2425MHz2427MHz_delay255us_tx120B_block645us.jsonl'}
{'transmitter': 'BLE1MBit', 'blocker': 'tone', 

In [33]:
import jsonlines
import os

def load_reference_message(reference_folder, transmitter_filename):
    # Construct the full path to the reference message file
    file_path = os.path.join(reference_folder, transmitter_filename)

    with jsonlines.open(file_path) as reader:
        reference_message = reader.read()["message"]
    
    return reference_message
    

def count_bit_errors(reference_message, message_bytes):
    # Initialize error counts
    bit_error_counts = [0] * (len(reference_message) * 8)  # For each bit in the message

    # Ensure messages are of equal length for comparison
    if len(reference_message) != len(message_bytes):
        raise ValueError("Message lengths do not match!")
    
    # Calculate bit errors by comparing each byte of the reference message and received message
    for byte_index, (ref_byte, msg_byte) in enumerate(zip(reference_message, message_bytes)):
        # Perform XOR operation between the reference byte and the message byte
        error_bits = ref_byte ^ msg_byte

        # Count bit errors by checking each bit position in the error_bits
        for bit_index in range(8):  # There are 8 bits in a byte
            if error_bits & (1 << bit_index):
                # Increment error count for the specific bit position
                bit_error_counts[byte_index * 8 + bit_index] += 1

    # Return the total number of bit errors
    total_bit_errors = sum(bit_error_counts)
    return total_bit_errors, bit_error_counts

def read_jsonl_file_and_calculate_bit_errors(file_path, reference_message):
    # Initialize total bit errors and cumulative bit error counts
    total_bit_errors = 0
    cumulative_bit_error_counts = [0] * (len(reference_message) * 8)  # Initialize cumulative counts for all bit positions

    with jsonlines.open(file_path) as reader:
        for line in reader:
            message_bytes = line["message"]
            
            # Calculate bit errors with respect to the reference message
            packet_bit_errors, bit_error_counts = count_bit_errors(reference_message, message_bytes)

            # Add to the total bit errors and accumulate the bit error counts
            total_bit_errors += packet_bit_errors
            for i in range(len(cumulative_bit_error_counts)):
                cumulative_bit_error_counts[i] += bit_error_counts[i]

    return total_bit_errors, cumulative_bit_error_counts

# Example usage:
reference_folder = "reference_messages"
transmitter_filename = "BLE1MBit.jsonl"  # Replace with your actual transmitter file name

# Load reference message from the corresponding transmitter file
reference_message = load_reference_message(reference_folder, transmitter_filename)

# Example: Read and calculate errors from a JSONL file
file_path = "experiment_data/BLE1MBit_IEEE802154250Kbit_0dBm-4dBm_2425MHz2423MHz_delay255us_tx120B_block8B.jsonl"
total_bit_errors, cumulative_bit_error_counts = read_jsonl_file_and_calculate_bit_errors(file_path, reference_message)

# Output the result
print(f"Total bit errors: {total_bit_errors}")
print(f"Cumulative bit error counts for each bit position: {len(cumulative_bit_error_counts)}")


Total bit errors: 0
Cumulative bit error counts for each bit position: 960
